## Construcción del archivo base para visualización de mortalidad infantil por desnutrición

#### Importación de librerias

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import mapclassify
import geopandas as gpd


#### Importacion de las bases de datos

Los microdatos se consiguen en el siguiente link:
https://microdatos.dane.gov.co/index.php/catalog/DEM-Microdatos

Ahi se descargan las Estadisticas Vitales y se usan los csv's "defunciones no fetales", los cuales fueron guardados en una carpeta llamada Defunciones


Para entender las variables es necesario tener el diccionario. Este mismo lo pueden encontrar en el siguiente link:
https://microdatos.dane.gov.co/index.php/catalog/696/pdf-documentation

In [ ]:
df1= pd.read_csv("Defunciones/nofetal2019.csv",encoding="latin1")
df2=pd.read_csv("Defunciones/nofetal2020.csv", encoding="latin1")
df3=pd.read_csv("Defunciones/nofetal2021.csv",encoding="latin1")
df4=pd.read_csv("Defunciones/nofetal2022.csv",encoding="latin1")
df5=pd.read_csv("Defunciones/nofetal2023.csv",encoding="latin1")

#### Selección de variables requeridas

In [ ]:
defun19=df1[['COD_DPTO','COD_MUNIC','CODPTORE','CODMUNRE','ANO', 'GRU_ED1', 'EST_CIVIL','SEXO','CAU_HOMOL', 'CAUSA_667']]
defun20=df2[['COD_DPTO','COD_MUNIC','CODPTORE','CODMUNRE','ANO', 'GRU_ED1', 'EST_CIVIL','SEXO','CAU_HOMOL', 'CAUSA_667']]
defun21=df3[['COD_DPTO','COD_MUNIC','CODPTORE','CODMUNRE','ANO', 'GRU_ED1', 'EST_CIVIL','SEXO','CAU_HOMOL', 'CAUSA_667']]
defun22=df4[['COD_DPTO','COD_MUNIC','CODPTORE','CODMUNRE','ANO', 'GRU_ED1', 'EST_CIVIL','SEXO','CAU_HOMOL', 'CAUSA_667']]
defun23=df5[['COD_DPTO','COD_MUNIC','CODPTORE','CODMUNRE','ANO', 'GRU_ED1', 'EST_CIVIL','SEXO','CAU_HOMOL', 'CAUSA_667']]


#### Concatenacion de los dataframes

In [ ]:
defu=pd.concat([defun19,defun20,defun21,defun22,defun23], ignore_index=True)


### 🔍 Filtrado de muertes por desnutrición y deficiencias nutricionales

Para este análisis, se filtraron las defunciones cuya causa agrupada corresponde al código **602** de la **Lista 6/67 de la OPS**, adoptada por el DANE para la clasificación de causas de muerte. Esta categoría incluye los códigos **E40 a E46** y **E50 a E64** de la **Clasificación Internacional de Enfermedades (CIE-10)**, que comprenden:

- **Desnutrición energético-proteica** (como marasmo, kwashiorkor y desnutrición severa no especificada),
- **Deficiencias de vitaminas y minerales**, como vitamina A, complejo B, hierro, entre otros.

Este filtro permite aislar los casos asociados a **deficiencias nutricionales**, incluyendo **anemias nutricionales**, sin mezclar otras causas de mortalidad infantil.

### Grupo Etario

- **Grupos etarios menores de 5 años**: Se seleccionaron únicamente los registros correspondientes a niños y niñas **entre 0 y 4 años**, que comprenden la primera infancia. Esta etapa de la vida es crítica desde el punto de vista nutricional, ya que la desnutrición en estos años tiene efectos severos sobre el desarrollo físico y cognitivo, además de una mayor vulnerabilidad frente a enfermedades.

Con esta doble condición se garantiza que el análisis refleje con mayor precisión el comportamiento de la mortalidad por desnutrición **en la población infantil más vulnerable**, lo que es fundamental para generar evidencia útil en la formulación de políticas públicas en salud y nutrición.

### NA's

Se eliminaron los registros con informacion ausente para hacer el dataset más optimo


In [ ]:
nutri = defu[(defu['CAUSA_667'] == 602) & (defu['GRU_ED1'] <= 8)]
nutri=nutri.dropna()
nutri

### 🧩 Construcción del código DIVIPOLA

Para poder enlazar la información de defunciones con el mapa municipal, es necesario contar con un identificador geográfico estándar. En este caso se construyó el campo `MUNICIPIO`, correspondiente al **código DIVIPOLA del MUNICIPIO**, concatenando:

- El **código del departamento de residencia**, ajustado a 2 dígitos con ceros a la izquierda.
- El **código del MUNICIPIO de residencia**, ajustado a 3 dígitos con ceros a la izquierda.

> ⚠️ **Importante**: Se utilizó la **residencia habitual del menor** (y no el lugar donde ocurrió el fallecimiento), con el objetivo de identificar los territorios donde se concentra la mayor vulnerabilidad y donde deben focalizarse las acciones de política pública.

Este identificador garantiza la compatibilidad con los archivos espaciales (shapefiles) oficiales del DANE, donde los municipios están identificados mediante el código DIVIPOLA de 5 dígitos.


In [ ]:
nutri[['CODPTORE','CODMUNRE']]= nutri[['CODPTORE','CODMUNRE']].astype(int)
nutri['CODPTORE']=nutri['CODPTORE'].astype(str).str.zfill(2)
nutri['CODMUNRE']=nutri['CODMUNRE'].astype(str).str.zfill(3)
nutri['MUNICIPIO'] = nutri['CODPTORE'] + nutri['CODMUNRE']

### Agrupación de  muertes infantil por desnutricion  por año y municipio de residencia

En este paso se agrupa el total de casos de desnutricion por año (`ANO`) y por código de MUNICIPIO  de residencia (`MUNICIPIO`), utilizando el método `groupby()` seguido de `size()` para contar los registros.

Luego, se renombran las columnas resultantes para mayor claridad:
- `MUNICIPIO` se cambia a `MPIO`
- `ANO` se cambia a `AÑO`

Esto permite tener una tabla resumen con el número de muertes por desnutrición (0 a 4 años) registrados por MUNICIPIO de residencia y por año, que puede usarse para análisis estadístico o para representar la información gráficamente (por ejemplo, en mapas o series temporales).

In [ ]:
tabla= nutri.groupby(['ANO','MUNICIPIO']).size().reset_index(name="Cantidad")
tabla= tabla.rename(columns={'MUNICIPIO':'MPIO', 'ANO':'AÑO'})

### Importacion de proyecciones de población DANE

#### Proyección de población por municipio - 2019

##### 🧮 Procesamiento de la población menor de 5 años

Este fragmento de código tiene como objetivo calcular la población total de menores de 5 años por municipio para el año 2019. A continuación, se describen los pasos realizados:

1. **Carga del archivo**  
   Se lee el archivo de proyecciones de población (`DCD_19_munic.xlsx`), omitiendo las primeras 11 filas que contienen metadatos.

2. **Filtrado por año y área geográfica**  
   Se filtran únicamente los registros correspondientes al año **2019** y al total del país (`"Total"`), es decir, sin distinción entre zonas urbanas y rurales.

3. **Estandarización del código municipal**  
   Se convierte el código del municipio (`MPIO`) a cadena de texto de 5 dígitos para facilitar la unión con otras fuentes de datos.

4. **Eliminación de columnas no necesarias**  
   Se eliminan las columnas `ÁREA GEOGRÁFICA` y `DPNOM`, que no son requeridas para el análisis posterior.

5. **Transformación de formato ancho a largo**  
   Se reorganiza el DataFrame usando `melt`, lo cual permite convertir las columnas de edad (`Total_0`, `Total_1`, etc.) en una sola columna llamada `Edad`.

6. **Selección de los grupos etarios deseados**  
   Se filtran únicamente las edades `Total_0` a `Total_4`, que corresponden a la población menor de 5 años.

7. **Agregación por municipio**  
   Se agrupan los datos por municipio (`MPIO`) y año (`AÑO`), sumando la población de las edades seleccionadas.

8. **Ajuste de tipos de datos**  
   Finalmente, se convierten las columnas `Población_menor` y `AÑO` a tipo entero (`int`) para asegurar consistencia en el análisis posterior.


In [ ]:
pob_19=pd.read_excel("Poblaciones/DCD_19_munic.xlsx",skiprows=11)
pob_19=pob_19[(pob_19['AÑO']==2019) & (pob_19['ÁREA GEOGRÁFICA']=="Total")]
pob_19['MPIO']=pob_19['MPIO'].astype(str).str.zfill(5)
pob_19.drop(columns=['ÁREA GEOGRÁFICA','DPNOM'], inplace=True)
pob_1=pob_19.melt(id_vars=['MPIO', 'AÑO'], var_name='Edad', value_name='Población_menor')
pob_1=pob_1[pob_1['Edad'].isin(['Total_0', 'Total_1', 'Total_2', 'Total_3', 'Total_4'])]
pob_1=pob_1.groupby(['MPIO', 'AÑO'])['Población_menor'].sum().reset_index()
pob_1[['Población_menor','AÑO']] = pob_1[['Población_menor','AÑO']].astype(int)

#### Proyección de población por departamento - 2020 a 2023

### Proyección de población por municipio - 2020 a 2023

##### 🧮 Procesamiento de la población menor de 5 años

Este fragmento de código tiene como objetivo calcular la población total de menores de 5 años por municipio para los años **2020 a 2023**. A continuación, se describen los pasos realizados:

1. **Carga del archivo**  
   Se lee el archivo de proyecciones de población (`DCD_20_23_munic.xlsx`), omitiendo las primeras 8 filas que contienen metadatos.

2. **Filtrado por años y área geográfica**  
   Se seleccionan únicamente los registros correspondientes a los años **2020, 2021, 2022 y 2023** y al total nacional (`"Total"`), es decir, sin diferenciar entre zonas urbanas y rurales.

3. **Estandarización del código municipal**  
   Se convierte el código del municipio (`MPIO`) a tipo entero para corregir posibles errores y luego a cadena de texto de 5 dígitos para facilitar su uso posterior.

4. **Transformación de formato ancho a largo**  
   Se reorganiza el DataFrame usando `melt`, lo cual convierte las columnas de edad (`Total_0`, `Total_1`, etc.) en una sola columna llamada `Edad`.

5. **Selección de los grupos etarios deseados**  
   Se filtran únicamente las edades `Total_0` a `Total_4`, correspondientes a la población menor de 5 años.

6. **Agregación por municipio y año**  
   Se agrupan los datos por municipio (`MPIO`) y año (`AÑO`), sumando la población de las edades seleccionadas.

7. **Ajuste de tipos de datos**  
   Finalmente, se convierten las columnas `Población_menor` y `AÑO` a tipo entero (`int`) para asegurar consistencia en el análisis posterior.



In [ ]:
pob_20_23=pd.read_excel("Poblaciones/DCD_20_23_munic.xlsx",skiprows=8)
pob_20_23=pob_20_23[(pob_20_23['AÑO'].between(2020, 2023)) & (pob_20_23['ÁREA GEOGRÁFICA']=="Total")]
pob_20_23['MPIO']=pob_20_23['MPIO'].astype(int)
pob_20_23['MPIO']=pob_20_23['MPIO'].astype(str).str.zfill(5)
pob_2=pob_20_23.melt(id_vars=['MPIO', 'AÑO'], var_name='Edad', value_name='Población_menor')
pob_2=pob_2[pob_2['Edad'].isin(['Total_0', 'Total_1', 'Total_2', 'Total_3', 'Total_4'])]
pob_2=pob_2.groupby(['MPIO', 'AÑO'])['Población_menor'].sum().reset_index()
pob_2[['Población_menor','AÑO']] = pob_2[['Población_menor','AÑO']].astype(int)


### Unificación de proyecciones de población (2019 - 2023)

Una vez procesadas por separado las proyecciones de población del año 2019 (`pob_1`) y las de los años 2020 a 2023 (`pob_2`), es necesario combinarlas en un único DataFrame para facilitar su uso posterior en los cálculos de tasas de mortalidad infantil por desnutricion por municipio y año.

Se utiliza la función `pd.concat()` con `ignore_index=True` para:
- Unir verticalmente los dos DataFrames (es decir, apilar las filas uno debajo del otro).
- Reiniciar los índices del nuevo DataFrame unificado.

El resultado es una tabla completa con la población menor a 5 años proyectada para todos los municipios colombianos entre 2019 y 2023, lo cual permitirá calcular tasas comparables y consistentes en todo el período analizado.




In [ ]:
pob= pd.concat([pob_1,pob_2], ignore_index=True)

### Cálculo de la tasa de mortalidad infantil por desnutricion por municipio y año

Una vez unificados los datos de población (`pob`) y el conteo de muertes por desnutrición en niños menores de 5 años por municipio y año (`tabla`), se realiza una fusión con `pd.merge()` utilizando un `left join`. Esto garantiza que todos los registros de población se conserven, incluso si no hay registros para ese año o municipio.

In [ ]:
data=pd.merge(pob,tabla, how="left", on=['AÑO', 'MPIO'])
#se reemplazan los valores nulos en la columna Cantidad (ausencia de muertes por desnutrición reportados) por ceros
data['Cantidad']=data['Cantidad'].fillna(0)
# se convierte la columna Cantidad a tipo entero
data['Cantidad']=data['Cantidad'].astype(int)
# se calcula la tasa de muerte infantil por desnutricion por cada 100.000 infantes menores de 5 años
data['tasa']=round(data['Cantidad']/data['Población_menor']*100000,2)

#### Guardar como csv para usar en la animación

In [ ]:
data.to_csv("tasa_mortalidad_menores_5.csv", index=False)